In [2]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=st121413&password=[password]&submit=Login' https://www.cityscapes-dataset.com/login/
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=1

--2021-02-25 03:43:43--  https://www.cityscapes-dataset.com/login/
Connecting to 192.41.170.23:3128... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://www.cityscapes-dataset.com/downloads/ [following]
--2021-02-25 03:43:45--  https://www.cityscapes-dataset.com/downloads/
Reusing existing connection to www.cityscapes-dataset.com:443.
Proxy request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [  <=>               ]  43.80K   189KB/s    in 0.2s    

2021-02-25 03:43:45 (189 KB/s) - ‘index.html’ saved [44855]

--2021-02-25 03:43:46--  https://www.cityscapes-dataset.com/file-handling/?packageID=1
Connecting to 192.41.170.23:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 252567705 (241M) [application/octet-stream]
Saving to: ‘gtFine_trainvaltest.zip’

gtFine_trainvaltest 100%[===================>] 240.87M  1.30MB/s    in 2m 36s  

2021-02-25 03:46:23 (1.55 MB/s) -

In [1]:
from torchvision import datasets
# https://pytorch.org/vision/0.8/datasets.html#cityscapes
# split (string, optional) – The image split to use, train, test or val if mode=”fine” otherwise train, train_extra or val
# dataset = datasets.Cityscapes('./data/cityscapes', split='train', mode='fine', target_type='semantic')
target_type = ['instance', 'semantic', 'polygon', 'color']
dataset = datasets.Cityscapes('./data/cityscapes', split='val', mode='fine', target_type=target_type)

RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory